In [38]:
import sys
sys.path.insert(0, '..')
print(sys.path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
print(sys.path)
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import json

['..', '..', '..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.ipython', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/tmp/tmpzyzywg49', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model']
['..', '..', '..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/u

In [39]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference/test.csv")
clean_df.head()

,Unnamed: 0,test_caption,test_imgs
0,0,feiyangg/LP Paragraph Style Electric Guitar Ti...,O1CN01r2RqZk1lJ4Sk2ZYaN_!!2200643934797.jpg
1,1,Summer Lace White Shorts Pants Female Thin Ant...,O1CN01H0o2pi2HAtGC9rVWa_!!2206736519111.jpg
2,2,High-Quality Export 8MM with Adhesive Cork Cor...,O1CN01qeHW0Z1D3mRPfmNvM_!!161-0-lubanu.jpg
3,3,LED Ceiling Living Room Bedroom Corridor Corri...,O1CN01phQoOe25ljjN1lpLO_!!3082327567.jpg
4,4,Disposable Cotton Cotton Female Thin Water Fac...,O1CN01Dzpgpr2FxJsUzqAdE_!!2963188946.jpg


In [40]:
image_embed_path = "./image_embeds"
text_embed_path = "./text_embeds"
image_embed_files = os.listdir(image_embed_path)
text_embed_files = os.listdir(text_embed_path)

In [41]:
text_embed_files

['txt_emded-base-base_1st_e_5.pkl',
 'txt_emded-base-base_3rd_e_5.pkl',
 'txt_emded-base-base_2nd_e_5.pkl',
 'txt_emded-base-base_all_e_7.pkl',
 'txt_emded-large-large_all_e_9.pkl',
 'txt_emded-large-large_v1_e_5.pkl']

In [42]:
all_image_embed ={}
all_text_embed = {}
for i,t in tqdm(zip(image_embed_files,text_embed_files)):
    with open(f"{image_embed_path}/{i}","rb") as img_file:
        img_embed = pickle.load(img_file)
        all_image_embed[img_embed["model_name"]] = img_embed["embed"]
    with open(f"{text_embed_path}/{t}","rb") as txt_file:
        txt_embed = pickle.load(txt_file)
        all_text_embed[txt_embed["model_name"]] = txt_embed["embed"]


6it [00:00,  6.35it/s]


In [43]:
all_image_embed.keys()

dict_keys(['base_all_e_7', 'base_2nd_e_5', 'base_1st_e_5', 'large_v1_e_5', 'base_3rd_e_5', 'large_all_e_9'])

In [44]:
all_text_embed.keys()

dict_keys(['base_1st_e_5', 'base_3rd_e_5', 'base_2nd_e_5', 'base_all_e_7', 'large_all_e_9', 'large_v1_e_5'])

In [45]:
pair_to_test = "base_3rd_e_5"

In [46]:
image_embed = all_image_embed[pair_to_test]
text_embed = all_text_embed[pair_to_test]

In [47]:
# test  =text_embed @ image_embed.T

In [48]:
# torch.topk(test,10)

In [49]:

def get_preds(all_image_embed,all_text_embed,pair_to_test):
    image_embed = all_image_embed[pair_to_test]
    text_embed = all_text_embed[pair_to_test]
    faiss_index = faiss.IndexFlatIP(256)
    faiss_index.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
    
    return top5_k_y_pred,top10_k_y_pred,top5_k_e,top10_k_e

In [50]:
top5_pred_idx,top10_pred_idx,top5_k_e,top10_k_e = get_preds(all_image_embed,all_text_embed,pair_to_test)

In [51]:
def remap_pred_to_name(top5_pred_idx,top10_pred_idx,image_array):
    top5_pred = [list(image_array[p]) for p in top5_pred_idx]
    top10_pred =  [list(image_array[p]) for p in top10_pred_idx]
    return top5_pred,top10_pred

In [52]:
top5_pred,top10_pred =remap_pred_to_name(top5_pred_idx,top10_pred_idx,clean_df["test_imgs"].values)

In [53]:
def results_to_json(top5_pred,top10_pred,caption_array,result_dir,result_file_name):
    results = []
    for c,t5,t10 in tqdm(zip(caption_array,top5_pred,top10_pred)):
        result_dict = {}
        result_dict["caption"] = c
        result_dict["top5"] = t5
        result_dict["top10"] = t10
        results.append(result_dict)
    print("checking results format")
    print(results[0])
    with open(f"{result_dir}/{result_file_name}.json","w") as results_file:
        json.dump(results,results_file)
    with open(f"{result_dir}/{result_file_name}.json","r") as results_file:
        check_results = json.load(results_file)
    print("checking results format of saved file")
    print(check_results[0])
    return check_results
    
        



In [54]:
saved_result = results_to_json(top5_pred,top10_pred,clean_df["test_caption"].values[0:20000],"./submissions","test_single_b_e_5_v3_3_6_22")

20000it [00:00, 672131.79it/s]


checking results format
{'caption': 'feiyangg/LP Paragraph Style Electric Guitar Tiger Veneer Factory Direct Color Can Be Customized', 'top5': ['O1CN011pW88rTaqfnNIi1_!!754885367.jpg', 'TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg', 'O1CN016zxVN01qPYIlq8the_!!0-item_pic.jpg', 'O1CN01h0ZgaM1qPYIr3q9Cf_!!0-item_pic.jpg', 'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png'], 'top10': ['O1CN011pW88rTaqfnNIi1_!!754885367.jpg', 'TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg', 'O1CN016zxVN01qPYIlq8the_!!0-item_pic.jpg', 'O1CN01h0ZgaM1qPYIr3q9Cf_!!0-item_pic.jpg', 'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png', 'O1CN01ha825u1ctPVv8XwDP_!!2211470413658-0-picasso.jpg', 'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg', 'O1CN01S8ewME1m1UwiK89Pr_!!1973994894.jpg', 'TB2lylSaQ.OyuJjSszhXXbZbVXa_!!20295181.jpg', 'O1CN01XJH6iU1lKuBgk0ndT_!!2207916444801.jpg']}
checking results format of saved file
{'caption': 'feiyangg/LP Paragraph Style Electric Guitar Tiger Veneer Factory Direct Color Can Be Customized', 'top5': 

In [56]:
len(saved_result)

20000

In [63]:
saved_result[0]

{'caption': 'feiyangg/LP Paragraph Style Electric Guitar Tiger Veneer Factory Direct Color Can Be Customized',
 'top5': ['O1CN011pW88rTaqfnNIi1_!!754885367.jpg',
  'TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg',
  'O1CN016zxVN01qPYIlq8the_!!0-item_pic.jpg',
  'O1CN01h0ZgaM1qPYIr3q9Cf_!!0-item_pic.jpg',
  'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png'],
 'top10': ['O1CN011pW88rTaqfnNIi1_!!754885367.jpg',
  'TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg',
  'O1CN016zxVN01qPYIlq8the_!!0-item_pic.jpg',
  'O1CN01h0ZgaM1qPYIr3q9Cf_!!0-item_pic.jpg',
  'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png',
  'O1CN01ha825u1ctPVv8XwDP_!!2211470413658-0-picasso.jpg',
  'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg',
  'O1CN01S8ewME1m1UwiK89Pr_!!1973994894.jpg',
  'TB2lylSaQ.OyuJjSszhXXbZbVXa_!!20295181.jpg',
  'O1CN01XJH6iU1lKuBgk0ndT_!!2207916444801.jpg']}